In [0]:
import pandas as pd

def textfeatures(features):
	phrase=[]
	for feature in list(features):
		words = ""
# 		print("feature",feature,type(feature))
		for word in feature.split(","):
			word = word.replace(" ","")
			word = word.replace("'","")
			word = word.replace("[","")
			word = word.replace("]","")
			words += word + " "
# 			print("word",word)
		phrase.append(words)
# 		break
	return phrase

if __name__ == "__main__":
	data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/final_features.csv").fillna("undefined")
	labels = data["author"]
	text = data["text"]
	## text features
	nouns = data["nouns"]
	named_ents = data["named_ents"]
	verbs = data["verb"]
	adjs = data["adj"]
	noun_phrases = data["noun_phrases"]

	## count features
	count_nouns = data["noun_tokens"]
	count_named_ents = data["entity_tokens"]
	count_verbs = data["verb_tokens"]
	count_adjs = data["adj_tokens"]
	count_noun_phrases = data["noun_phrases_tokens"]

	## extra count features
	unique_words = data["num_unique_words"]
	stopwords = data["num_stopwords"]
	articles = data["article"]
	punc = data["num_punctuations"]
	mean_word_len = data["mean_word_len"]
	non_voc = data["n_non_voc"]

	noun_phrases = textfeatures(noun_phrases)
	nouns = textfeatures(nouns)
	verbs = textfeatures(verbs)
	named_ents = textfeatures(named_ents)
	adjs = textfeatures(adjs)


Vectorisation of features

In [0]:
# TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
# text = pd.DataFrame(tfidf.fit_transform(text).toarray())
nouns = pd.DataFrame(tfidf.fit_transform(nouns).toarray())
# verbs = pd.DataFrame(tfidf.fit_transform(verbs).toarray())
# named_ents = pd.DataFrame(tfidf.fit_transform(named_ents).toarray())
# adjs = pd.DataFrame(tfidf.fit_transform(adjs).toarray())
# noun_phrases = pd.DataFrame(tfidf.fit_transform(noun_phrases).toarray())

## ordinal encoder
from sklearn.preprocessing import OrdinalEncoder
labels = pd.DataFrame(OrdinalEncoder().fit_transform(labels.to_numpy().reshape(-1,1)))
print("encoded")

## joining
# X = pd.concat([count_nouns,count_named_ents,count_verbs,count_adjs,count_noun_phrases,
#               unique_words,stopwords,articles,punc,mean_word_len,non_voc,
#               nouns,named_ents,verbs])
# print("joined",X.shape)

## de allocating memory
# nouns = named_ents = verbs = adjs = count_nouns = count_named_ents = count_verbs = count_adjs = count_noun_phrases = unique_words = stopwords = articles = punc = mean_word_len = non_voc = 0
print("de allocation")

## train test split
from sklearn.model_selection import train_test_split
steps = 25000
X_train,X_test,Y_train,Y_test = train_test_split(nouns, labels, test_size=0.1,shuffle = True)


encoded
de allocation


Training a model


SVM

In [0]:
## training a model
# from sklearn.svm import SVC
# svm = SVC(kernel = "linear",C=0.025, random_state = 13)
# svm.fit(X_train,Y_train)
# y_pred = svm.predict(X_test)

Decision Tree


In [0]:
# from sklearn.linear_model import RidgeClassifier
# clf = RidgeClassifier()
# clf.fit(X_train,Y_train)
# Y_predict = clf.predict(X_test)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:940: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



LBGM

In [0]:
import lightgbm as lgb
mdl = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=63, max_depth=-1, learning_rate=0.1, n_estimators=100)
mdl.fit(X_train, Y_train.values.ravel())
Y_predict = mdl.predict(X_test)


In [0]:
## confusion matrix

from sklearn.metrics import mean_squared_error,confusion_matrix,accuracy_score,precision_recall_fscore_support
confMatrix = confusion_matrix(Y_test,Y_predict)
accuracy_noun = accuracy_score(Y_test,Y_predict)
precision_recall_fscore = precision_recall_fscore_support(Y_test,Y_predict)

print("[nouns] Accurracy {} Precision recall {} confusion matrix {} ".format(accuracy_noun,precision_recall_fscore,confMatrix))

[nouns] Accurracy 0.503216583273767 Precision recall (array([0.53050109, 0.61466165, 0.56047198, 0.31940299]), array([0.63001294, 0.59025271, 0.61889251, 0.24970828]), array([0.57599054, 0.60220994, 0.58823529, 0.28028815]), array([773, 554, 614, 857])) confusion matrix [[487  30  61 195]
 [ 73 327  23 131]
 [ 80  24 380 130]
 [278 151 214 214]] 


Pure test SET


In [0]:
## testing previous set
from sklearn.metrics import mean_squared_error,confusion_matrix,accuracy_score,precision_recall_fscore_support
Y_test_predict = clf.predict(nouns.iloc[steps:,:])
Y_test_final = labels.iloc[steps:,:]

confMatrix = confusion_matrix(Y_test_final,Y_test_predict)
accuracy_noun = accuracy_score(Y_test_final,Y_test_predict)
precision_recall_fscore = precision_recall_fscore_support(Y_test_final,Y_test_predict)


print("[nouns][TEST] Accurracy {} Precision recall {} confusion matrix {} ".format(accuracy_noun,precision_recall_fscore,confMatrix))